In [2]:
from pathlib import Path
import json
import re
from tqdm import tqdm

settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['LM(한국어)', '문서 내용 QA', '답변 근거 생성', 'Multiple-choice', '표 QA', '질문 생성']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)
    
#### prepare for task preprocess



#### prepare for task preprocess end

#### task preprocess
for split in splits:
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        file_task = source_data.stem.split('_')[1] #multiple, span, tableqa, text, unanswerable
        source_data = source_data.open()
        for line in source_data.readlines():
            line = json.loads(line)

            #### LM(한국어)
            data = {'text': None}
            ## preprocess data from line
            text = ''
            for cqa in line['paragraphs']:
                text += '##context:\n'+cqa['context']+ '\n'
                for qa in cqa['qas']:
                    text += '##question:\n'+qa['question']+ '\n'
                    if qa['is_impossible']:
                        text += '##answer:\n' + 'unanswerable.\n'
                    else:
                        text += '##answer:\n' + qa['answers']['text'] + '\n'
            data['text'] = text+'\n'
            ## preprocess data from line end
            task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### LM(한국어) end
            
            
            for paragraph in line['paragraphs']:
                for qa in paragraph['qas']:
                    #### data preprocess
                    data = {'input': None, 'output': None}
                    data['input'] = {}
                    data['input']['doc_id'] = line['doc_id']
                    data['input']['context_id'] = paragraph['context_id']
                    data['input']['context'] = paragraph['context']
                    data['input']['question_id'] = qa['question_id']
                    data['input']['question'] = qa['question']
                    data['output'] = {}
                    data['output']['answer'] = qa['answers']['text']
                                            
                    if file_task == 'tableqa':    
                        #### 표 QA
                        task_files[4].write(json.dumps(data, ensure_ascii=False)+'\n')
                        # print(json.dumps(data, indent=4, ensure_ascii=False))
                        #### 표 QA end
                    
                    data['output']['answer_start'] = qa['answers']['answer_start']
                    data['output']['clue_text'] = qa['answers']['clue_text']
                    data['output']['clue_start'] = qa['answers']['clue_start']
                    #### data preprocess end 
                    if file_task in ['span', 'text', 'unanswerable']:
                        #### 문서 내용 QA
                        task_files[1].write(json.dumps(data, ensure_ascii=False)+'\n')
                        # print(json.dumps(data, indent=4, ensure_ascii=False))
                        #### 문서 내용 QA end
                
                    elif file_task == 'multiple':
                        #### Multiple-choice
                        data['input']['choices'] = qa['answers']['options']
                        
                        task_files[3].write(json.dumps(data, ensure_ascii=False)+'\n')
                        # print(json.dumps(data, indent=4, ensure_ascii=False))
                        #### Multiple-choice end

            
                    if data['output']['clue_text']:
                        #### 답변 근거 생성
                        ## preprocess data from line
                        data['input']['answer'] = data['output'].pop('answer')
                        data['output'].pop('answer_start')
                        ## preprocess data from line end
                        task_files[2].write(json.dumps(data, ensure_ascii=False)+'\n')
                        # print(json.dumps(data, indent=4, ensure_ascii=False))
                        #### 답변 근거 생성 end
            
            
                    #### 질문 생성
                    ## preprocess data from line
                    data['input'].update(data['output'])
                    data['input'].pop('question_id')
                    data['output'] = {'question' : data['input'].pop('question')}
                    ## preprocess data from line end
                    task_files[5].write(json.dumps(data, ensure_ascii=False)+'\n')
                    # print(json.dumps(data, indent=4, ensure_ascii=False))
                    #### 질문 생성 end
            
            
            
            # break
        # break
    # break

    for path in task_files:
        path.close()      
        
#### task preprocess end      

valid: 100%|██████████| 6/6 [00:02<00:00,  2.95it/s]
